<sub>Felix Becker, Lars Gabriel University of Greifswald, Germany</sub>
# Exercise Set 1 - 1

The potential of machine learning ultimatively comes from pattern detection in large amounts of data. However, before we can train a model and make predictions, we have to:
 - Analyse the raw data (which filetypes, how is the data organized?)
 - Read data file(s) into memory
 - Make statistical analyses (e.g. plots)  
 - Manipulate the data (reshape and mathematical operations), remove, add or combine features
 
## Raw data

Inspect the file *bikes-summerdays-full.csv*. How many datapoints and attributes are there? What delimiter is used? Has the file one or more header rows? Are there missing values?

*Tip: Double clicking the file in jupyter opens a CSVTable view. You can also open the file as raw text by right clicking $\rightarrow$ Open With $\rightarrow$ Editor.*

## Read the file



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In the cell below, read the data file as a pandas dataframe called ``df``. Take care to handle the header line of the file correctly and use the right delimiter/separator.

In [2]:
df = pd.read_csv("bikes-summerdays-full.csv", sep = ) #YOUR TASK: choose a correct separator with the "sep" argument

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-2-f4b6fa5f8588>, line 1)

Print the following things:
- What is the shape of the dataframe?
- What are the datatypes of the columns "season" and "temp"?
- What is the maximum and minimum temperature?

Remember that `df` is a [pandas dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) and that you can use class member functions to solve the task. Alternatively, you could also apply numpy functions to numeric columns. Useful functions are `np.amin` and `np.amax`.

In [ ]:
## YOUR CODE HERE
## play around with the dataframe and complete the tasks above

The dataset contains dates that are rather useless if stored as strings:

In [ ]:
df["datetime"]

The following cell will convert *datetime* to a proper time format that allows the extraction of hours, days and month. If the following cell prints an error message, go back and make sure you chose the correct separator character.

In [ ]:
df["datetime"] = pd.DatetimeIndex(df["datetime"])
df["hour"] = [t.hour for t in df["datetime"]]
df["dayofweek"] = [t.dayofweek for t in df["datetime"]]
df["month"] = [t.month for t in df["datetime"]]

## Getting warm: Simple statistical analyses

The original data came with a description for each data column. Lets assume we don't have this description and want to find out what the encoding 1,2,3,4 for "season" means. 

- Print (a subset of) the values from the "season" column
- Use the helper function below to create boxplots for the temperatures of the 4 seasons and make a guess which number corresponds to which season.

*Tips (assuming X,Y are vectors of the same size):*
- *``X == i`` returns a boolean vector that has "true" on each position that matches i.*
- *use ``Y[X == i]`` (boolean mask) to retrieve subsets*

In [ ]:
# expects data to be a list of length 4, where entry i is a np.array of temperatures for season i
def boxplots(data):
    fig, ax = plt.subplots()
    ax.set_title("Temperature by season.")
    ax.set_xlabel("seasons")
    ax.set_ylabel("temperature in celsius")
    ax.boxplot(data)
    
# YOUR CODE STARTS
data = [np.ones(10)]*4 #replace this line
# YOUR CODE ENDS

boxplots(data)

## Feature engineering: Create a datamatrix

Most of the time, the raw data that we read from a file is not sufficient to be directly fed to a ML model. Usually we do:
- normalize continuous variables (in some form) 
- one hot represent discrete variables
- apply other transformations, for example we could guess that both for very low and very high temperatures fewer bikes are rented so that the square of the temperature might be useful

*Note that the last point becomes less important with modern deep learning as in principal non-linear models could learn these transformations on their own.*

Create a `np.matrix` $X$ of datatype float with the following columns:
- workingday, holiday
- normalized: windspeed, humidity, temp, atemp
- squares of temp and atemp
- one hot represented (!) weather and season
- hour, dayofweek, month (normalized to (0,1], e.g. lunch time at 12am should map to 0.5)

A **one hot representation** for example of the four seasons would map season $2$ to the vector $[0,0,1,0].$

*Tip: Use the functions `to_one_hot` and `normalize`*

In [ ]:
m = df.shape[0] #number of datapoints
n = 19 #number of attributes

X = np.zeros((m, n)) # initialize with all zeros

#assumes that classes is a vector of length m with elements in range 0,1,...,num_classes-1
#returns a matrix of shape (m, num_classes)
def to_one_hot(classes, num_classes):
    return np.eye(num_classes)[classes]

#assumes that X is a vector, returns a normalized X
def normalize(X):
    return (X - np.mean(X)) / np.std(X)

#example:
X[:,0] = df["workingday"]

# YOUR CODE STARTS
# add the remaining columns X[:,1], X[:,2], ...
# YOUR CODE ENDS

X

In [ ]:
#save X to a (binary) file and use it later in another notebook
np.save("X", X)
#for convenience, also save y
np.save("y", df["count"])

We will use $X$ later to train a neural network.

In Python *numpy* is usually used to transfer numeric data between frameworks e.g. it allows communication of *pandas* and *tensorflow*.

## Fit a multivariate regression model

Fitting a linear regression model on our prepared data using high-level keras is very easy. Read and execute the code below and see it happening.

In [9]:
import tensorflow as tf

In [ ]:
#construct the model which is just ja single Dense layer with 1 output unit
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units = 1))

#here we use the popular Adam optimizer (a variant of stochastic gradient descent)
#it's typically a very good choice for a broad range of problems
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1)

#we compile the model and tell the framework to use mean squared error
model.compile(optimizer=optimizer, loss="mse")

#fit the model for 10 epochs (i.e. each datapoint is seen 10 times) on 80% of the data
#test the models performance on the other 20% of the data
history = model.fit(x=X, y=y, epochs=10, validation_split=0.2)

In [ ]:
def plot_error(err, label, col="red"):
    ''' Plot error over time'''
    plt.plot(np.log(err), 'o-', color=col, linewidth=.2, markersize=3, mfc='none', label=label)
    plt.title("Log error by iteration of gradient descent")
    plt.xlabel('iteration '+r'$i$')
    plt.ylabel(r'$ln \;E(\theta^{(i)})$');
    
plt.subplots()
plot_error(history.history["loss"], label="train error")
plot_error(history.history["val_loss"], col="blue", label="test error")
plt.legend()